Furkan Çetin - 16/11/2020

# Project #1: Finding Lane Lines on the Road (for images)

## Main Code

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os

In [ ]:
#Listing images 
list_images_names = os.listdir("test_images/")

for img_name in list_images_names:
    img_input = cv2.imread(r'test_images/'+img_name)
    img_output = process_image(img_input)
    plt.imshow(img_output[...,::-1])
    plt.show()
    cv2.imwrite(r'test_images_output/Output_'+img_name, img_output)

## process_image(image_input)

In [ ]:
def process_image(image_input):
    # x coordinate of vertical line located at the center 
    mid_vertical_x = 485
    
    # point list of the polygonal area on image where we want to apply our filters
    region_coordinates =[[(110,540), (950,540), (550,330), (420,330)]]   

    # Convert image into gray scale
    image_grey = cv2.cvtColor(image_input,cv2.COLOR_RGB2GRAY)
    
    # Add images extra blurring
    image_grey_blurred = cv2.GaussianBlur(image_grey,(5, 5),0)
    
    # Apply Canny algorithm for edge detection 
    image_canny = cv2.Canny(image_grey_blurred, 40, 100)

    # Crop the images with polygonal region
    mask = np.zeros_like(image_canny)    
    vertices = np.array(region_coordinates, dtype=np.int32)
    cv2.fillPoly(mask, vertices, 255)
    image_masked_edges = cv2.bitwise_and(image_canny, mask)

    # Apply Hough Transformations
    lines = cv2.HoughLinesP(image_masked_edges, 2, (np.pi)/180, 15, np.array([]),40, 20)
  
    # Draw final lines onto the image
    xLeft  = []
    yLeft = []
    xRight  = []
    yRight = []  
    
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            # check for the FIRST point of HoughLines list 
            if (x1 < mid_vertical_x): # that means the point is on the LEFT lane
                xLeft.append(x1)
                yLeft.append(y1)
            else:                     # that means the point is on the RIGHT lane
                xRight.append(x1)
                yRight.append(y1)
                    
            # check for the SECOND point of HoughLines list       
            if (x2 < mid_vertical_x): # that means the point is on the LEFT lane
                xLeft.append(x2)
                yLeft.append(y2)
            else:                     # that means the point is on the RIGHT lane
                xRight.append(x2)
                yRight.append(y2)

    
    # Find average of all the points on both lanes and fit a line onto them
    blank_canvas = np.zeros(image_input.shape, np.uint8)

    [slope, intercept] = np.polyfit(xLeft, yLeft, 1)
    startY = 330
    endY = 540
    startX = int( (startY - intercept) / slope)
    endX = int((endY - intercept) / slope)
    cv2.line(blank_canvas,(startX,startY),(endX,endY),(0, 0, 255) ,11) # Red Line for LEFT Lane  [(B,G,R) standart for OpenCV]
    
    [slope, intercept] = np.polyfit(xRight, yRight, 1)
    startY = 330
    endY = 540
    startX = int( (startY - intercept) / slope)
    endX = int((endY - intercept) / slope)
    cv2.line(blank_canvas,(startX,startY),(endX,endY),(0, 0, 255) ,11)  # Red Line for RIGHT Lane  [(B,G,R) standart for OpenCV]
    
    image_result = cv2.addWeighted(image_input, 1.0, blank_canvas, 0.5, 0)
    
    # Return the final image that consists the input image with detected lane lines
    return image_result